In [1]:
# !pip install --user --upgrade catboost
# !pip install --user --upgrade ipywidgets
# !pip install shap
# !pip install sklearn
# !pip install --upgrade numpy
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
!pip install --pre deepchem
!pip install --user --upgrade catboost
!pip install --user --upgrade ipywidgets
!pip install shap
!pip install sklearn
!pip install --upgrade numpy
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
import deepchem as dc
import pandas as pd
from deepchem.molnet.load_function.molnet_loader import TransformerGenerator, _MolnetLoader

from deepchem.data import Dataset
from typing import List, Optional, Tuple, Union
import pandas as pd
import numpy as np
import random
import os
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import catboost
from catboost import *
from catboost import datasets
from catboost import CatBoostClassifier

/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py:119: PkgResourcesDeprecationWarning: 0.18ubuntu0.18.04.1 is an invalid version and will not be supported in a future release
  PkgResourcesDeprecationWarning,


In [4]:

import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
import catboost
from catboost import *
from catboost import datasets


In [5]:
!pwd
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
import catboost
from catboost import *
from catboost import datasets

/Solution/cp


In [15]:
task = "/Solution/cp/train_cp.csv"

feature_field="smiles"
TASKS = ['The Carcinogenic Potency Database (CPDB)_Carcinogenic Potency Expressed']
task_dir = "/Solution/Ames/"
task_save = "/Solution/Ames/"
df = pd.read_csv(task)
class MyLoader(_MolnetLoader):

    def create_dataset(self) -> Dataset:
        loader = dc.data.CSVLoader(tasks=self.tasks,
                                   feature_field=feature_field,
                                   featurizer=self.featurizer)
        return loader.create_dataset(task, shard_size=8192)


def load_tox21(
    featurizer: Union[dc.feat.Featurizer, str] = 'ECFP',
    splitter: Union[dc.splits.Splitter, str, None] = 'scaffold',
    transformers: List[Union[TransformerGenerator, str]] = ['balancing'],
    reload: bool = True,
    data_dir: Optional[str] = task_dir,
    save_dir: Optional[str] = task_save,
    **kwargs
) -> Tuple[List[str], Tuple[Dataset, ...], List[dc.trans.Transformer]]:
    
    loader = MyLoader(featurizer, splitter, transformers, TASKS,
                          data_dir, save_dir, **kwargs)
    return loader.load_dataset(task, reload)
tasks, datasets, transformers = load_tox21()
train_dataset, valid_dataset, test_dataset = datasets
df_tr = train_dataset.to_dataframe()
df_t = test_dataset.to_dataframe()
df_v = valid_dataset.to_dataframe()

In [18]:

x = df_tr.drop(['y','w','ids'],axis=1)
y = df_tr['y']
xt = df_t.drop(['y','w','ids'],axis=1)
 
x_smiles = df_t['ids'].to_numpy()

yt = df_t['y']
xv = df_v.drop(['y','w','ids'],axis=1)

yv = df_v['y']

In [19]:

x = df_tr.drop(['y','w','ids'],axis=1)
y = df_tr['y']
xt = df_t.drop(['y','w','ids'],axis=1)
x_smiles = df_t['ids'].to_numpy()

yt = df_t['y']
xv = df_v.drop(['y','w','ids'],axis=1)

yv = df_v['y']

In [20]:
def set_seed(s):
    seed_value= s
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    import random
    random.seed(seed_value)
    import numpy as np
    np.random.seed(seed_value)



In [21]:
set_seed(17)

In [22]:
import matplotlib.pyplot as plt
def show_history(hist):
    acc = hist.history['AUC']
    val_acc = hist.history['val_AUC']
    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, '-', label='AUC')
    plt.plot(epochs, val_acc, ':', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.plot()

In [23]:
cat_features = list(range(x.shape[1]))

In [24]:
type(cat_features[0])

int

In [25]:
pool = Pool(data=x, label=y)


In [26]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=50,
    learning_rate=0.00001,
    loss_function='CrossEntropy',
    eval_metric='AUC',
    
  
)

model.fit(
    x, y,
    # cat_features=cat_features,
    eval_set=(xv, yv),
    verbose=False,
    plot=True,

)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Model is fitted: True
Model params:
{'eval_metric': 'AUC', 'loss_function': 'CrossEntropy', 'learning_rate': 1e-05, 'iterations': 50}


In [27]:
roc_auc_score(yt,model.predict_proba(xt)[:,1])

0.516729797979798

In [28]:
pred = model.predict(xt).astype('int64')
df_savey = pd.DataFrame(data={'smiles':list(x_smiles)})
df_savey['y']=pred
df_savey.to_csv('output_classification_cp_cat.csv',index=False)    